<center>

<p style="font-size:30px;"><strong> Computational Photonics </p>

<p style="font-size:30px;"><strong> Homework 2: Implementation of the Beam Propagation Method </p>


</center>

<center>

**Author:**
*Group 1*
| Name             | Email       |
| -----------      | ----------- |
| *Kiril Armstrong*|  *kiril.armstrong@uni-jena.de*   |
| *Lena Fleischmann*   |  *lena.fleischmann@uni-jena.de*           |
| *Md Zobaer Ahmed Rahat*  |  *@uni-jena.de*           |
| *Yucheng Sun*     |  *yucheng.sun@uni-jena.de*        |

</center>

>**Supervisor:**
>
>*Prof. Thomas Pertsch* 
>
>**Tutor:**
>
>*Tobias Bucher*
>
>*Jan Sperrhake*


In [ ]:
import numpy as np
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
from matplotlib import pyplot as plt
# from Homework_2_function_headers import waveguide, gauss, beamprop_FN, beamprop_CN, beamprop_BN

# dark bluered colormap, registers automatically with matplotlib on import
import bluered_dark

plt.rcParams.update({
        'figure.figsize': (12/2.54, 9/2.54),
        'figure.subplot.bottom': 0.15,
        'figure.subplot.left': 0.165,
        'figure.subplot.right': 0.925,
        'figure.subplot.top': 0.9,
        'axes.grid': True,
})
plt.close('all')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

**Table of contents**<a id='toc0_'></a>    
- 1. [Introduction](#toc1_)    
- 2. [Beam Propagation Method](#toc2_)    
- 3. [Von Neumann stability analysis](#toc3_)    
- 4. [Analysis and Simulation of the Problems](#toc4_)    
  - 4.1. [Task 1 - explicit - implicit scheme](#toc4_1_)    
    - 4.1.1. [Implementation](#toc4_1_1_)    
    - 4.1.2. [Convergence Tests](#toc4_1_2_)    
    - 4.1.3. [Example](#toc4_1_3_)    
  - 4.2. [Task 2 - explicit scheme](#toc4_2_)    
    - 4.2.1. [Implementation](#toc4_2_1_)    
    - 4.2.2. [Convergence Tests](#toc4_2_2_)    
    - 4.2.3. [Example](#toc4_2_3_)    
  - 4.3. [Task 3 - implicit scheme](#toc4_3_)    
    - 4.3.1. [Implementation](#toc4_3_1_)    
    - 4.3.2. [Convergence Tests](#toc4_3_2_)    
    - 4.3.3. [Example](#toc4_3_3_)    
- 5. [Conclusion](#toc5_)    
- 6. [References](#toc6_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Introduction](#toc0_)

In this project, we employ the beam propagation method to identify the stationary field distribution of a step-index slab waveguide with a Gaussian initial field distribution in 2D (x-z plane). And we use *Python* to implement this method. In addition, both the physical properties and the numerical properties (i.e. the convergence test of the discretization in z direction and also in x direction) of the simulation will be discussed.

## 2. <a id='toc2_'></a>[Beam Propagation Method](#toc0_)

The beam propagation method is developed to solve the problem in which the index distribution changes weakly along the propagation direction. 

In this method, the most crucial approximation is the slowly varying envelope approximation. When considering an electromagnetic wave in a homogeneous space, it can be demonstrated that the propagating wave is solely a function of the propagation direction and time, and thus has only transverse components. If weak changes are introduced along the propagation direction, the transverse fields will also undergo slight changes along the two transverse directions, resulting in a longitudinal component due to the divergence condition of the field. Based on these, we can replace the quickly varying component, $\Phi$, with a slowly varying one $\phi$, as

\begin{align}
\Phi(x, y, z)=\phi(x, y, z)\exp(-ikn_0z), \tag{}
\end{align}

where $n_0$ is the reference index.



## 3. <a id='toc3_'></a>[Von Neumann stability analysis](#toc0_)

## 4. <a id='toc4_'></a>[Analysis and Simulation of the Problems](#toc0_)

- Waveguide profile

In [ ]:
def waveguide(xa, xb, Nx, n_cladding, n_core):
    '''Generates the refractive index distribution of a slab waveguide
    with step profile centered around the origin of the coordinate
    system with a refractive index of n_core in the waveguide region
    and n_cladding in the surrounding cladding area.
    All lengths have to be specified in µm.

    Parameters
    ----------
        xa : float
            Width of calculation window
        xb : float
            Width of waveguide
        Nx : int
            Number of grid points
        n_cladding : float
            Refractive index of cladding
        n_core : float
            Refractive index of core

    Returns
    -------
        n : 1d-array
            Generated refractive index distribution
        x : 1d-array
            Generated coordinate vector
    '''
    
    x = np.linspace(-xa/2, xa/2, Nx)
    n = np.zeros(Nx, dtype=float)
    # index distribution
    for i in range(Nx):
        if abs(x[i]) <= xb/2:
            n[i] = n_core
        else:
            n[i] = n_cladding
    # for i, xi in enumerate(x):
    #     if abs(xi) <= xb/2:
    #         n[i] = n_core
    #     else:
    #         n[i] = n_cladding
    return n, x

- Initial field - Gaussian field distribution

In [ ]:
def gauss(xa, Nx, w):
    '''Generates a Gaussian field distribution v = exp(-x^2/w^2) centered
    around the origin of the coordinate system and having a width of w.
    All lengths have to be specified in µm.

    Parameters
    ----------
        xa : float
            Width of calculation window
        Nx : int
            Number of grid points
        w  : float
            Width of Gaussian field

    Returns
    -------
        v : 1d-array
            Generated field distribution
        x : 1d-array
            Generated coordinate vector
    '''
    
    x = np.linspace(-xa/2, xa/2, Nx)
    v = np.exp(-x**2/w**2)
    return v, x

- Basic parameters

In [ ]:
# computational parameters
z_end   = 100       # propagation distance
lam     = 1         # wavelength
nd      = 1.455     # reference index
xa      = 50        # size of computational window

# waveguide parameters
xb      = 2.0       # size of waveguide
n_cladding  = 1.45      # cladding index
n_core  = 1.46      # core refr. index

# source width
w       = 5.0       # Gaussian beam width

### 4.1. <a id='toc4_1_'></a>[Task 1 - explicit - implicit scheme](#toc0_)

#### 4.1.1. <a id='toc4_1_1_'></a>[Implementation](#toc0_)

In [ ]:
def beamprop_CN(v_in, lam, dx, n, nd,  z_end, dz, output_step):
    '''Propagates an initial field over a given distance based on the
    solution of the paraxial wave equation in an inhomogeneous
    refractive index distribution using the explicit-implicit
    Crank-Nicolson scheme. All lengths have to be specified in µm.

    Parameters
    ----------
        v_in : 1d-array
            Initial field
        lam : float
            Wavelength
        dx : float
            Transverse step size
        n : 1d-array
            Refractive index distribution
        nd : float
            Reference refractive index
        z_end : float
            Propagation distance
        dz : float
            Step size in propagation direction
        output_step : int
            Number of steps between field outputs

    Returns
    -------
        v_out : 2d-array
            Propagated field
        z : 1d-array
            z-coordinates of field output
    '''
    
    # Basic parameters - wavenumbers
    k0 = 2*np.pi/lam
    kd = nd*k0
    k1 = n*k0
    k2 = np.ones(len(k1)) * kd

    # Construction of the operator matrix L1
    ## Diagonal elements
    diagonals_1 = np.zeros((3, len(n)))
    diagonals_1[0] = np.ones(len(n)) * (-2)
    diagonals_1[1] = np.ones(len(n)) * 1
    diagonals_1[-1] = np.ones(len(n)) * 1
    diag_position_1 = [0, 1, -1]
    ## Sparse matrix construction
    L1 = sps.diags(diagonals_1, diag_position_1)
    L1 = (1j/(2*kd*dx**2)) * L1

    # Construction of the operator matrix L2
    ## Diagonal elements
    diagonals_2 = np.zeros((1, len(n)))
    diagonals_2[0] = (k1**2 - k2**2) / (2*kd)
    diag_position_2 = [0]
    ## Sparse matrix construction
    L2 = sps.diags(diagonals_2, diag_position_2)
    L2 = 1j*L2

    # Construction of the operator matrix L
    L = L1 + L2

    # Crank-Nicolson scheme
    # Do not consider output_step
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(z), len(n)), dtype=complex)
    # for i in range(len(z)):
    #     ## Construction of the operator matrix M1
    #     M1 = sps.eye(len(n)) - (z[i]/2) * L
    #     ## Construction of the operator matrix M2
    #     M2 = sps.eye(len(n)) + (z[i]/2) * L

    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[i,:] = sps.linalg.spsolve(M1, M2.dot(v_in))

    # Consider output_step with delta_z = z[i]
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(range(0, len(z), output_step)), len(n)), dtype=complex)
    # counter = 0
    # for i in range(0, len(z), output_step):
    #     ## Construction of the operator matrix M1
    #     M1 = sps.eye(len(n)) - (z[i]/2) * L
    #     ## Construction of the operator matrix M2
    #     M2 = sps.eye(len(n)) + (z[i]/2) * L

    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[counter,:] = sps.linalg.spsolve(M1, M2.dot(v_in))

    #     counter += 1

    # Consider output_step with delta_z = dz
    z = []
    z.append(0)
    v_out = []
    v_out.append(v_in)
    counter = 1
    i = 0
    for i in range(int(z_end/(dz*output_step)) + 1):
        if z[i] > z_end:
            break
        M1 = sps.eye(len(n)) - (dz/2) * L
        M2 = sps.eye(len(n)) + (dz/2) * L
        v_out.append(sps.linalg.spsolve(M1, M2.dot(v_out[counter - 1][:])))
        z.append(z[i] + dz*output_step)
        i += 1
        counter += 1

    return v_out, z

#### 4.1.2. <a id='toc4_1_2_'></a>[Convergence Tests](#toc0_)

1. Convergence test for dz

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size
output_step = 1     # output step size


# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# propagation step size - 31 points logarithmically spaced between 10 and 0.053
# dz = np.logspace(np.log10(1), np.log10(0.05), 31)
# dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.8, 1.0]
# dz = [0.01, 0.0125, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.078125, 0.1, 0.125, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 1.0]
dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 0.8, 1.0]

operation_time = np.zeros(len(dz))
field_end = np.zeros((len(dz), Nx))
# dz_num = []
counter = 0
for i, dzi in enumerate(dz):
    start = time.time()
    v_out, z = beamprop_CN(v_in, lam, dx, n, nd,  z_end, dzi, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    print("dz = %6.6f, time = %gs" % (dzi, stop - start))
    field_end[counter][:] = np.abs(v_out[-1][:])**2
    counter += 1
    # z = z[::output_step]
    # if abs(z[-1] - z_end) < 1e-6:
    #    field_end[counter, :] = np.abs(v_out[-1, :])**2
    #    dz_num.append(dzi)
    #    counter += 1

# calculate relative error to the value obtained at highest resolution
# field_end = field_end[np.any(field_end != 0, axis=1)]
real_error = []
for i in range(field_end.shape[0]):
    real_error.append(np.abs(np.linalg.norm(field_end[0][:] - field_end[i][:]) / np.linalg.norm(field_end[0][:])))
    # real_error.append(np.sum(np.abs(field_end[1][:] - field_end[i][:])) / np.sum(np.abs(field_end[-1][:])))


# Plot of operation time
plt.figure()
plt.plot(dz, operation_time, 'o--')
plt.xlabel('dz [µm]')
plt.ylabel('operation time [s]')
plt.title('Operation time for different dz \n Crank-Nicolson scheme')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(len(dz)):
    plt.plot(x, field_end[i], label='dz = %.6f' % dz[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different dz \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(dz, real_error, 'o--')
plt.xlabel('dz [µm]')
plt.ylabel('relative error')
plt.title('Relative error for different dz \n Crank-Nicolson scheme')
# plt.xscale('log')
plt.yscale('log')
plt.show()

2. Convergence test for Nx

In [ ]:
# propagation step size
dz = 0.5
output_step = 1


Nx = np.linspace(151, 2151, 101, dtype=int)
operation_time = np.zeros(len(Nx))
field_end = []
x_end = []
for i, Nxi in enumerate(Nx):
    dx      = xa/(Nxi-1) # transverse step size
    start = time.time()
    # create index distribution
    n, x = waveguide(xa, xb, Nxi, n_cladding, n_core)
    # create initial field
    v_in, x     = gauss(xa, Nxi, w)
    v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity
    # propagate field
    v_out, z = beamprop_CN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    x_end.append(x)
    field_end.append(np.abs(v_out[-1][:])**2)
    # print("Nx = %s, time = %gs" % (Nxi, stop - start))

# calculate relative error to the value obtained at highest resolution
real_error = np.zeros(len(Nx))
for i in range(len(Nx)):
    real_error[i] = np.abs(1 - np.linalg.norm(field_end[i])/np.linalg.norm(field_end[-1]))



# Plot of operation time
plt.figure()
plt.plot(Nx, operation_time, 'o-')
plt.xlabel('Nx')
plt.ylabel('operation time [s]')
plt.title('Operation time for different Nx \n Crank-Nicolson scheme')
# plt.xscale('log')
# plt.yscale('log')
plt.show()

# Plot results - x direction
plt.figure()
counter = 0
for i in range(len(field_end)):
    if counter > len(field_end):
        break
    plt.plot(x_end[counter], field_end[counter], label='Nx = %d' % Nx[counter])
    counter += 10
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different Nx \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(Nx, real_error, 'o-')
plt.xlabel('Nx')
plt.ylabel('relative error')
plt.title('Relative error for different Nx \n Crank-Nicolson scheme')
# plt.xscale('log')
# plt.yscale('log')
plt.show()

#### 4.1.3. <a id='toc4_1_3_'></a>[Example](#toc0_)

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size

# propagation step size
dz = 0.5
output_step = round(1.0/dz)

# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# calculation
v_out, z = beamprop_CN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
# z = z[::output_step]
for i in range(len(z)):
    v_out[i] = v_out[i]/np.sqrt(np.sum(np.abs(v_out[i])**2)) # normalize power to unity

# Plot results - x-z plane
plt.figure()
plt.pcolormesh(x, z, np.abs(v_out)**2, cmap='bluered_dark')
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('z [µm]')
plt.title('Field intensity distribution in the x-z plane of the waveguide \n Crank-Nicolson scheme')
plt.gca().set_aspect('equal')
cb = plt.colorbar()
cb.set_label('intensity')
plt.show()


# Plot results - x direction
plt.figure()
for i in range(0, len(z), 2):
    plt.plot(x, np.abs(v_out[i])**2, label='z = %d' % z[i])
# plt.plot(x, np.abs(v_out[0])**2, label='z = 0')
# plt.plot(x, np.abs(v_out[2*output_step])**2, label='z = z[2]')
# plt.plot(x, np.abs(v_out[6*output_step])**2, label='z = z[6]')
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution in the x direction at different z values \n Crank-Nicolson scheme')
plt.legend()
plt.show()

### 4.2. <a id='toc4_2_'></a>[Task 2 - explicit scheme](#toc0_)

#### 4.2.1. <a id='toc4_2_1_'></a>[Implementation](#toc0_)

In [ ]:
def beamprop_FN(v_in, lam, dx, n, nd,  z_end, dz, output_step):
    '''Propagates an initial field over a given distance based on the
    solution of the paraxial wave equation in an inhomogeneous
    refractive index distribution using the explicit scheme. All lengths have to be specified in µm.

    Parameters
    ----------
        v_in : 1d-array
            Initial field
        lam : float
            Wavelength
        dx : float
            Transverse step size
        n : 1d-array
            Refractive index distribution
        nd : float
            Reference refractive index
        z_end : float
            Propagation distance
        dz : float
            Step size in propagation direction
        output_step : int
            Number of steps between field outputs

    Returns
    -------
        v_out : 2d-array
            Propagated field
        z : 1d-array
            z-coordinates of field output
    '''
    # Basic parameters - wavenumbers
    k0 = 2*np.pi/lam
    kd = nd*k0
    k1 = n*k0
    k2 = np.ones(len(k1)) * kd

    # Construction of the operator matrix L1
    ## Diagonal elements
    diagonals_1 = np.zeros((3, len(n)))
    diagonals_1[0] = np.ones(len(n)) * (-2)
    diagonals_1[1] = np.ones(len(n)) * 1
    diagonals_1[-1] = np.ones(len(n)) * 1
    diag_position_1 = [0, 1, -1]
    ## Sparse matrix construction
    L1 = sps.diags(diagonals_1, diag_position_1)
    L1 = (1j/(2*kd*dx**2)) * L1

    # Construction of the operator matrix L2
    ## Diagonal elements
    diagonals_2 = np.zeros((1, len(n)))
    diagonals_2[0] = (k1**2 - k2**2) / (2*kd)
    diag_position_2 = [0]
    ## Sparse matrix construction
    L2 = sps.diags(diagonals_2, diag_position_2)
    L2 = 1j*L2

    # Construction of the operator matrix L
    L = L1 + L2

    # Explicit scheme
    # Do not consider output_step
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(z), len(n)), dtype=complex)
    # for i in range(len(z)):
    #     ## Construction of the operator matrix M
    #     M = sps.eye(len(n)) + (z[i]/2) * L

    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[i,:] = M2.dot(v_in)

    # Consider output_step with delta_z = z[i]
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(range(0, len(z), output_step)), len(n)), dtype=complex)
    # counter = 0
    # for i in range(0, len(z), output_step):
    #     ## Construction of the operator matrix M
    #     M = sps.eye(len(n)) + (z[i]) * L

    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[counter,:] = M.dot(v_in)
    #     counter += 1

    # Consider output_step with delta_z = dz
    z = []
    z.append(0)
    v_out = []
    v_out.append(v_in)
    counter = 1
    i = 0
    for i in range(int(z_end/(dz*output_step)) + 1):
        if z[i] > z_end:
            break
        # M1 = sps.eye(len(n))
        M2 = sps.eye(len(n)) + (dz) * L
        v_out.append(M2.dot(v_out[counter - 1][:]))
        z.append(z[i] + dz*output_step)
        i += 1
        counter += 1


    return v_out, z

#### 4.2.2. <a id='toc4_2_2_'></a>[Convergence Tests](#toc0_)

1. Convergence test for dz

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size
output_step = 1     # output step size

# propagation step size - 31 points logarithmically spaced between 10 and 0.053
# dz = np.logspace(np.log10(1), np.log10(0.05), 31)
# dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.8, 1.0]
dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 0.8, 1.0]

operation_time = np.zeros(len(dz))
field_end = np.zeros((len(dz), Nx))
# dz_num = []
counter = 0
for i, dzi in enumerate(dz):
    start = time.time()
    v_out, z = beamprop_FN(v_in, lam, dx, n, nd,  z_end, dzi, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    print("dz = %6.3f, time = %gs" % (dzi, stop - start))
    field_end[counter][:] = np.abs(v_out[-1][:])**2
    counter += 1
    # z = z[::output_step]
    # if abs(z[-1] - z_end) < 1e-6:
    #     field_end[counter, :] = np.abs(v_out[-1, :])**2
    #     dz_num.append(dzi)
    #     counter += 1

# calculate relative error to the value obtained at highest resolution
real_error = []
for i in range(field_end.shape[0]):
    real_error.append(np.linalg.norm(field_end[0][:] - field_end[i][:]) / np.linalg.norm(field_end[0][:]))

# Plot of operation time
plt.figure()
plt.plot(dz, operation_time, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('operation time [s]')
plt.title('Operation time for different dz \n Explicit scheme')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(len(dz)):
    plt.plot(x, field_end[i], label='dz = %.6f' % dz[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different dz \n Explicit scheme')
plt.legend()
plt.show()


# Plot of relative error
plt.figure()
plt.plot(dz, real_error, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('relative error')
plt.title('Relative error for different dz \n Explicit scheme')
# plt.xscale('log')
plt.yscale('log')
plt.show()

2. Convergence test for Nx

In [ ]:
# propagation step size
dz = 0.5
output_step = 1

Nx = np.linspace(151, 1151, 101, dtype=int)
operation_time = np.zeros(len(Nx))
field_end = []
x_end = []
for i, Nxi in enumerate(Nx):
    dx      = xa/(Nxi-1) # transverse step size
    start = time.time()
    # create index distribution
    n, x = waveguide(xa, xb, Nxi, n_cladding, n_core)
    # create initial field
    v_in, x     = gauss(xa, Nxi, w)
    v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity
    # propagate field
    v_out, z = beamprop_FN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    x_end.append(x)
    field_end.append(np.abs(v_out[-1][:])**2)
    # print("Nx = %s, time = %gs" % (Nxi, stop - start))

# calculate relative error to the value obtained at highest resolution
real_error = np.zeros(len(Nx))
for i in range(len(Nx)):
    real_error[i] = np.abs(1 - np.linalg.norm(field_end[i])/np.linalg.norm(field_end[-1]))



# Plot of operation time
plt.figure()
plt.plot(Nx, operation_time, 'o-')
plt.xlabel('Nx')
plt.ylabel('operation time [s]')
plt.title('Operation time for different Nx \n Explicit scheme')
# plt.xscale('log')
# plt.yscale('log')
plt.show()

# Plot results - x direction
plt.figure()
counter = 0
for i in range(len(field_end)):
    if counter > len(field_end):
        break
    plt.plot(x_end[counter], field_end[counter], label='Nx = %d' % Nx[counter])
    counter += 5
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different Nx \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(Nx, real_error, 'o-')
plt.xlabel('Nx')
plt.ylabel('relative error')
plt.title('Relative error for different Nx \n Explicit scheme')
# plt.xscale('log')
# plt.yscale('log')
plt.show()

#### 4.2.3. <a id='toc4_2_3_'></a>[Example](#toc0_)

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size

# propagation step size
dz = 0.5
output_step = round(1.0/dz)

# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# calculation
v_out, z = beamprop_FN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
# z = z[::output_step]
for i in range(len(z)):
    v_out[i] = v_out[i]/np.sqrt(np.sum(np.abs(v_out[i])**2)) # normalize power to unity

# Plot results - x-z plane
plt.figure()
plt.pcolormesh(x, z, np.abs(v_out)**2, cmap='bluered_dark')
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('z [µm]')
plt.title('Field intensity distribution in the x-z plane of the waveguide \n Explicit scheme')
plt.gca().set_aspect('equal')
cb = plt.colorbar()
cb.set_label('intensity')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(0, len(z), 2):
    plt.plot(x, np.abs(v_out[i])**2, label='z = %d' % z[i])
# plt.plot(x, np.abs(v_out[0])**2, label='z = 0')
# plt.plot(x, np.abs(v_out[2*output_step])**2, label='z = 1')
# plt.plot(x, np.abs(v_out[6*output_step])**2, label='z = 2')
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution in the x direction at different z values \n Explicit scheme')
plt.legend()
plt.show()

### 4.3. <a id='toc4_3_'></a>[Task 3 - implicit scheme](#toc0_)

#### 4.3.1. <a id='toc4_3_1_'></a>[Implementation](#toc0_)

In [ ]:
def beamprop_BN(v_in, lam, dx, n, nd,  z_end, dz, output_step):
    '''Propagates an initial field over a given distance based on the
    solution of the paraxial wave equation in an inhomogeneous
    refractive index distribution using the implicit scheme. All lengths have to be specified in µm.

    Parameters
    ----------
        v_in : 1d-array
            Initial field
        lam : float
            Wavelength
        dx : float
            Transverse step size
        n : 1d-array
            Refractive index distribution
        nd : float
            Reference refractive index
        z_end : float
            Propagation distance
        dz : float
            Step size in propagation direction
        output_step : int
            Number of steps between field outputs

    Returns
    -------
        v_out : 2d-array
            Propagated field
        z : 1d-array
            z-coordinates of field output
    '''
    
    # Basic parameters - wavenumbers
    k0 = 2*np.pi/lam
    kd = nd*k0
    k1 = n*k0
    k2 = np.ones(len(k1)) * kd

    # Construction of the operator matrix L1
    ## Diagonal elements
    diagonals_1 = np.zeros((3, len(n)))
    diagonals_1[0] = np.ones(len(n)) * (-2)
    diagonals_1[1] = np.ones(len(n)) * 1
    diagonals_1[-1] = np.ones(len(n)) * 1
    diag_position_1 = [0, 1, -1]
    ## Sparse matrix construction
    L1 = sps.diags(diagonals_1, diag_position_1)
    L1 = (1j/(2*kd*dx**2)) * L1

    # Construction of the operator matrix L2
    ## Diagonal elements
    diagonals_2 = np.zeros((1, len(n)))
    diagonals_2[0] = (k1**2 - k2**2) / (2*kd)
    diag_position_2 = [0]
    ## Sparse matrix construction
    L2 = sps.diags(diagonals_2, diag_position_2)
    L2 = 1j*L2

    # Construction of the operator matrix L
    L = L1 + L2

    # Implicit scheme
    # Do not consider output_step
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(z), len(n)), dtype=complex)
    # for i in range(len(z)):
    #     ## Construction of the operator matrix M
    #     M = sps.eye(len(n)) - (z[i]) * L

    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[i,:] = sps.linalg.spsolve(M, v_in)

    # Consider output_step with delta_z = z[i]
    # z = np.linspace(0, z_end, int(z_end/dz) + 1)
    # v_out = np.zeros((len(range(0, len(z), output_step)), len(n)), dtype=complex)
    # counter = 0
    # for i in range(0, len(z), output_step):
    #     ## Construction of the operator matrix M
    #     M = sps.eye(len(n)) - (z[i]) * L

    #     # Solution of the slowly varying envelope along the propagation direction
    #     v_out[counter,:] = sps.linalg.spsolve(M, v_in)
    #     counter += 1

    # Consider output_step with delta_z = dz
    z = []
    z.append(0)
    v_out = []
    v_out.append(v_in)
    counter = 1
    i = 0
    for i in range(int(z_end/(dz*output_step)) + 1):
        if z[i] > z_end:
            break
        M1 = sps.eye(len(n)) - (dz) * L
        # M2 = sps.eye(len(n))
        v_out.append(sps.linalg.spsolve(M1, v_out[counter - 1][:]))
        z.append(z[i] + dz*output_step)
        i += 1
        counter += 1


    return v_out, z

#### 4.3.2. <a id='toc4_3_2_'></a>[Convergence Tests](#toc0_)

1. Convergence test for dz

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size
output_step = 1     # output step size

# propagation step size - 31 points logarithmically spaced between 10 and 0.053
# dz = np.logspace(np.log10(1), np.log10(0.05), 31)
# dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.8, 1.0]
# dz = [0.01, 0.0125, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.078125, 0.1, 0.125, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 1.0]
dz = [0.01, 0.0125, 0.016, 0.02, 0.025, 0.04, 0.05, 0.0625, 0.08, 0.1, 0.125, 0.16, 0.2, 0.25, 0.4, 0.5, 0.625, 0.78125, 0.8, 1.0]

operation_time = np.zeros(len(dz))
field_end = np.zeros((len(dz), Nx))
# dz_num = []
counter = 0
for i, dzi in enumerate(dz):
    start = time.time()
    v_out, z = beamprop_BN(v_in, lam, dx, n, nd,  z_end, dzi, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    print("dz = %6.3f, time = %gs" % (dzi, stop - start))
    field_end[counter][:] = np.abs(v_out[-1][:])**2
    counter += 1
    # z = z[::output_step]
    # if abs(z[-1] - z_end) < 1e-6:
    #     field_end[counter, :] = np.abs(v_out[-1, :])**2
    #     dz_num.append(dzi)
    #     counter += 1

# calculate relative error to the value obtained at highest resolution
real_error = []
for i in range(field_end.shape[0]):
    real_error.append(np.linalg.norm(field_end[0][:] - field_end[i][:]) / np.linalg.norm(field_end[0][:]))


# Plot of operation time
plt.figure()
plt.plot(dz, operation_time, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('operation time [s]')
plt.title('Operation time for different dz \n Implicit scheme')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(len(dz)):
    plt.plot(x, field_end[i], label='dz = %.6f' % dz[i])
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different dz \n Implicit scheme')
plt.legend()
plt.show()


# Plot of relative error
plt.figure()
plt.plot(dz, real_error, 'o-')
plt.xlabel('dz [µm]')
plt.ylabel('relative error')
plt.title('Relative error for different dz \n Implicit scheme')
# plt.xscale('log')
plt.yscale('log')
plt.show()


2. Convergence test for Nx

In [ ]:
# propagation step size
dz = 0.5
output_step = 1

Nx = np.linspace(151, 2151, 101, dtype=int)
operation_time = np.zeros(len(Nx))
field_end = []
x_end = []
for i, Nxi in enumerate(Nx):
    dx      = xa/(Nxi-1) # transverse step size
    start = time.time()
    # create index distribution
    n, x = waveguide(xa, xb, Nxi, n_cladding, n_core)
    # create initial field
    v_in, x     = gauss(xa, Nxi, w)
    v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity
    # propagate field
    v_out, z = beamprop_BN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
    stop = time.time()
    operation_time[i] = stop - start
    x_end.append(x)
    field_end.append(np.abs(v_out[-1][:])**2)
    # field_end.append(np.abs(v_out[-1]/np.sqrt(np.sum(np.abs(v_out[-1])**2)))**2)
    # print("Nx = %s, time = %gs" % (Nxi, stop - start))

# calculate relative error to the value obtained at highest resolution
real_error = np.zeros(len(Nx))
for i in range(len(Nx)):
    real_error[i] = np.abs(1 - np.linalg.norm(field_end[i])/np.linalg.norm(field_end[-1]))


# Plot of operation time
plt.figure()
plt.plot(Nx, operation_time, 'o-')
plt.xlabel('Nx')
plt.ylabel('operation time [s]')
plt.title('Operation time for different Nx \n Implicit scheme')
# plt.xscale('log')
# plt.yscale('log')
plt.show()

# Plot results - x direction
plt.figure()
counter = 0
for i in range(len(field_end)):
    if counter > len(field_end):
        break
    plt.plot(x_end[counter], field_end[counter], label='Nx = %d' % Nx[counter])
    counter += 10
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution at far end for different Nx \n Crank-Nicolson scheme')
plt.legend()
plt.show()

# Plot of relative error
plt.figure()
plt.plot(Nx, real_error, 'o-')
plt.xlabel('Nx')
plt.ylabel('relative error')
plt.title('Relative error for different Nx \n Implicit scheme')
# plt.xscale('log')
# plt.yscale('log')
plt.show()

#### 4.3.3. <a id='toc4_3_3_'></a>[Example](#toc0_)

In [ ]:
# transverse grid
Nx      = 251       # number of transverse points
dx      = xa/(Nx-1) # transverse step size

# propagation step size
dz = 0.5
output_step = round(1.0/dz)

# create index distribution
n, x = waveguide(xa, xb, Nx, n_cladding, n_core)

# create initial field
v_in, x     = gauss(xa, Nx, w)
v_in        = v_in/np.sqrt(np.sum(np.abs(v_in)**2)) # normalize power to unity

# calculation
v_out, z = beamprop_BN(v_in, lam, dx, n, nd,  z_end, dz, output_step)
# z = z[::output_step]
for i in range(len(z)):
    v_out[i] = v_out[i]/np.sqrt(np.sum(np.abs(v_out[i])**2)) # normalize power to unity

# Plot results - x-z plane
plt.figure()
plt.pcolormesh(x, z, np.abs(v_out)**2, cmap='bluered_dark')
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('z [µm]')
plt.title('Field intensity distribution in the x-z plane of the waveguide \n Implicit scheme')
plt.gca().set_aspect('equal')
cb = plt.colorbar()
cb.set_label('intensity')
plt.show()

# Plot results - x direction
plt.figure()
for i in range(0, len(z), 2):
    plt.plot(x, np.abs(v_out[i])**2, label='z = %d' % z[i])
# plt.plot(x, np.abs(v_out[0])**2, label='z = 0')
# plt.plot(x, np.abs(v_out[2*output_step])**2, label='z = 1')
# plt.plot(x, np.abs(v_out[6*output_step])**2, label='z = 2')
plt.axvline(x=-xb/2, color='r', linestyle='--')
plt.axvline(x=xb/2, color='r', linestyle='--')
plt.xlabel('x [µm]')
plt.ylabel('intensity')
plt.title('Field intensity distribution in the x direction at different z values \n Implicit scheme')
plt.legend()
plt.show()

## 5. <a id='toc5_'></a>[Conclusion](#toc0_)

## 6. <a id='toc6_'></a>[References](#toc0_)